In [2]:
class DataBase():
    '''
    Class for Databases
    '''
    pass

class SQL(DataBase):
    def __init__(self, nlquery, db):
        super().__init__(db)
        self.nlquery = nlquery
        self.db = db
    def __len__(self):
        return len(self.nlquery)
class ME(SQL):
    def __init__(self, nlquery):
        super().__init__(nlquery)
        self.element = None
    def search_model_entity(self, nlquery):
        '''
        Returns the number of ME if there is any, otherwise returns None
        '''
        pass
    def get_element(self, nlquery):
        # me = self.search_model_entity(nlquery)
        # if(me):
        #     return self.get_entity(me, nlquery,self.db)

        # return element else return None
        pass

        # SemSig = []
        # ParamSig = {}

In [144]:
SemSig = ["ME1","PVE1","ME2","PVE1","IVE1"]
ParamSig = {
    "ME1" : {
        "Element" : "ListedSecurity",
        "Property" : "RandomProperty" 
    },
    "IVE1" : {
        "Property" : "Corporation.hasLegalName",
        "Value" : ['ALPHABET HOLDING COMPANY, INC.', 'ALPHABET INC']
    },
    "ME2" : {
        "Element" : "MoneytaryAmount.hasAmount"
    },
    "PVE1" : {
        "Operand1" : "ME2",
        "Op" : ">",
        "Operand2" : "IVE1"
    },
    "Fetch" : {
        "FETCH" : "FETCH ALL"
    }
}

In [145]:
SemSig

['ME1', 'PVE1', 'ME2', 'PVE1', 'IVE1']

In [146]:
ParamSig

{'ME1': {'Element': 'ListedSecurity', 'Property': 'RandomProperty'},
 'IVE1': {'Property': 'Corporation.hasLegalName',
  'Value': ['ALPHABET HOLDING COMPANY, INC.', 'ALPHABET INC']},
 'ME2': {'Element': 'MoneytaryAmount.hasAmount'},
 'PVE1': {'Operand1': 'ME2', 'Op': '>', 'Operand2': 'IVE1'},
 'Fetch': {'FETCH': 'FETCH ALL'}}

In [149]:
sig1.dict

{'SELECT': {'ME1': {'Element': 'ListedSecurity', 'Property': 'RandomProperty'},
  'ME2': {'Element': 'MoneytaryAmount.hasAmount'}},
 'FROM': {'ME1': {'Element': 'ListedSecurity', 'Property': 'RandomProperty'},
  'ME2': {'Element': 'MoneytaryAmount.hasAmount'}},
 'WHERE': {'PVE1': {'Operand1': 'ME2', 'Op': '>', 'Operand2': 'IVE1'}},
 'SUBQUERIES': [<__main__.SQLSig at 0x7fc28568d550>],
 'FETCH': {'FETCH': 'FETCH ALL'}}

In [182]:
import copy
class SQLSig():
    def __init__(self, SemSig, ParamSig):
        self.SemSig = SemSig
        self.ParamSig = ParamSig
        self.subqueries = []
        self.dict = self.return_dict()
        if self.count_subqueries() > 0:
            for sub in range(1, self.count_subqueries()+1):
                SemSigUpd, ParamSigUpd = self.update_sig(sub)
                self.subqueries.append(SQLSig(SemSigUpd, ParamSigUpd))
    @staticmethod
    def dict_to_sig(dictionary):
        SemSig = []
        ParamSig = []
        for k, v in dictionary.items():
            if k in ["SUBQUERIES", "FETCH"]:
                continue
            for key, value in v.items():
                if key in SemSig:
                    continue
                SemSig.append(key)
                ParamSig.append(value)
        print(SemSig, ParamSig)
            
    def update_sig(self, idx):
        PVEParams = self.get_parameters(self.get_pve())
        wanted = PVEParams["PVE"+str(idx)]
        op1 = wanted["Operand1"]
        op2 = wanted["Operand2"]
        op = wanted["Op"]
        check = None
        if op1[:3] == "IVE":
            val_op1 = 2
            v = 1
        else:
            val_op1 = -2
        if op2[:3] == "IVE":
            val_op2 = 2
            v=2
        else:
            val_op2 = -2
        if op == ">":
            val_op = 1
        else:
            val_op = -1
        fin = val_op1*val_op*val_op2
        
        if fin == 4:
            agg = "MAX"
        elif fin == -4:
            agg = "MIN"
        
        # x = self.return_dict()
        SELECT =  copy.deepcopy(self.dict["SELECT"])
        FROM = copy.deepcopy(self.dict["FROM"])
        # "INNER JOIN" : self.get_parameters(self.get_ive()),
        WHERE = copy.deepcopy(self.dict["WHERE"])
        SUBQUERIES = copy.deepcopy(self.dict["SUBQUERIES"])
        FETCH = copy.deepcopy(self.dict["FETCH"])
        for me in self.get_entity():    
            if "." in SELECT[me]["Element"]:
                kisko = me
        SELECT[kisko]["Agg"] = agg
        # del WHERE["PVE"+str(idx)]
        WHERE = {key:val for key, val in WHERE.items() if key not in [f"PVE+{str(idx)}"]}
        # if v == 2:
        params = copy.deepcopy(self.get_parameters(self.get_ive()))["IVE"+str(idx)]
        # print(params)
        WHERE["PVE"+str(idx)]["Operand1"] = params["Value"]
        WHERE["PVE"+str(idx)]["Op"] = "IN"        
        WHERE["PVE"+str(idx)]["Operand2"] = params["Property"]
        

        # x = self.get_parameters(self.get_pve())["PVE"+str(idx)]

# Change ME to what we require subquery me (using its properties) [Choose which me to aggregate using its properties]
# Change IVE from PVE to something, so that subqueries dont go too many [Either this or change WHERE to IVE intead of PVE]
# Fetch all
# Where change to IVE instead of PVE
# Update ME

        # if v == 1:
        #     new_dict = {key:val for key, val in x.items() if key not in ['Operand1', 'Op']}

        #     # non_sub_op = op2
        # elif v == 2:
        #     new_dict = {key:val for key, val in x.items() if key not in ['Operand2', 'Op']}
        #     # x = self.get_parameters(self.get_pve())["PVE"+str(idx)]
        #     # non_sub_op = op1
        # #[non_sub_op]
        # print("x = ", x, "new_dict = ", new_dict)
        # if fin == -4:
        #     x["Agg"] = "MIN"
        # else:
        #     x["Agg"] = "MAX"
        # SemSig = [x, v]
        # ParamSig = {}
        # ParamSig = {
        #     x,
        #     v
        # }


        # print(SemSig, ParamSig)
        return [], {}

    def get_entity(self):
        ME = []
        for sem in self.SemSig:
            if sem[:2] == "ME":
                ME.append(sem)
        return ME
    def get_pve(self):
        PVE = []
        for sem in self.SemSig:
            if sem[:3] == "PVE":
                PVE.append(sem)
        return PVE
    def get_ive(self):
        IVE = []
        for sem in self.SemSig:
            if sem[:3] == "IVE":
                IVE.append(sem)
        return IVE

    def get_parameters(self, mode):
        Props = {}
        for term in mode:
            if term in self.ParamSig.keys():
                Props[term] = ParamSig[term]
        return Props
        
    def count_subqueries(self):
        IVE = get_ive(self.SemSig)
        return len(IVE)
    
    def get_fetch(self):
        try:
            FETCH = self.ParamSig["Fetch"]
        except:
            FETCH = "FETCH ALL"
        return FETCH

    def return_dict(self):
        SigDict = {
            "SELECT" : self.get_parameters(self.get_entity()),
            "FROM" : self.get_parameters(self.get_entity()),
            # "INNER JOIN" : self.get_parameters(self.get_ive()),
            "WHERE" : self.get_parameters(self.get_pve()),
            "SUBQUERIES" : self.subqueries,
            "FETCH" : self.get_fetch()
        }
        return SigDict
    def __str__(self):
        print(self.return_dict())
sig1 = SQLSig(SemSig, ParamSig)

In [183]:
sig1.dict

{'SELECT': {'ME1': {'Element': 'ListedSecurity', 'Property': 'RandomProperty'},
  'ME2': {'Element': 'MoneytaryAmount.hasAmount'}},
 'FROM': {'ME1': {'Element': 'ListedSecurity', 'Property': 'RandomProperty'},
  'ME2': {'Element': 'MoneytaryAmount.hasAmount'}},
 'WHERE': {'PVE1': {'Operand1': 'ME2', 'Op': '>', 'Operand2': 'IVE1'}},
 'SUBQUERIES': [<__main__.SQLSig at 0x7fc2853ef390>],
 'FETCH': {'FETCH': 'FETCH ALL'}}

In [184]:
sig1.return_dict()["SUBQUERIES"][0].return_dict()

{'SELECT': {}, 'FROM': {}, 'WHERE': {}, 'SUBQUERIES': [], 'FETCH': 'FETCH ALL'}

In [185]:
sig1.dict_to_sig(sig1.dict)

['ME1', 'ME2', 'PVE1'] [{'Element': 'ListedSecurity', 'Property': 'RandomProperty'}, {'Element': 'MoneytaryAmount.hasAmount'}, {'Operand1': 'ME2', 'Op': '>', 'Operand2': 'IVE1'}]


In [18]:
def get_entity(SemSig):
    ME = []
    for sem in SemSig:
        if sem[:2] == "ME":
            ME.append(sem)
    return ME
def get_pve(SemSig):
    PVE = []
    for sem in SemSig:
        if sem[:3] == "PVE":
            PVE.append(sem)
    return PVE
def get_ive(SemSig):
    IVE = []
    for sem in SemSig:
        if sem[:3] == "IVE":
            IVE.append(sem)
    return IVE

def check_sanity(obj):
    if len(obj) < 1:
        return None
    return obj

def get_parameters(term, ParamSig):
    Props = {}
    for term in term:
        if term in ParamSig.keys():
            Props[term] = ParamSig[term]
    return Props
    
def count_subqueries(SemSig):
    ive = get_ive(SemSig)
    return len(ive)

SigDict = {
    "SELECT" : get_parameters(get_entity(SemSig), ParamSig),
    "FROM" : get_parameters(get_entity(SemSig), ParamSig),
    "INNER JOIN" : get_parameters(get_ive(SemSig), ParamSig),
    "WHERE" : get_parameters(get_pve(SemSig), ParamSig),
    "SUBQUERIES" : count_subqueries(SemSig)
}